In [1]:
from fastai import *
from fastai.docs import *
from fastai.text import *

In [2]:
# untar_data(IMDB_PATH)

In [3]:
df = pd.read_csv(IMDB_PATH/'train.csv', header=None)
df.head()

,0,1
0,0,Un-bleeping-believable! Meg Ryan doesn't even ...
1,1,This is a extremely well-made film. The acting...
2,0,Every once in a long while a movie will come a...
3,1,Name just says it all. I watched this movie wi...
4,0,This movie succeeds at being one of the most u...


In [4]:
classes = read_classes(IMDB_PATH/'classes.txt')
classes[0], classes[1]

('negative', 'positive')

In [5]:
data_lm = text_data_from_csv(Path(IMDB_PATH), data_func=lm_data)
# data_lm = text_data_from_csv(Path(IMDB_PATH))

In [21]:
list(iter(data_lm.train_dl))

[[tensor([[  42, 2435,    0,  ...,   16,   21,   27],
          [  39,    9,   21,  ...,  819,    0,   11],
          [4612,    0,    6,  ...,   13,   16,   80],
          ...,
          [   9,    8,    3,  ...,   22,    2,  727],
          [   6,  404,   24,  ...,    5, 1913,   61],
          [ 595,   10,   13,  ..., 1239,  219,   30]], device='cuda:0'),
  tensor([  39,    9,   21,  ...,   10, 1182, 4604], device='cuda:0')],
 [tensor([[  88,   54,   17,  ...,   10, 1182, 4604],
          [  18,   23,   29,  ...,  184,    3,   27],
          [ 113,    0,    2,  ...,    3,  104,    5],
          ...,
          [   3,   31,   90,  ...,    2,    4,  905],
          [  19,  517,  260,  ...,  228, 5722,   25],
          [  24,   31,   21,  ...,   83,    5,    3]], device='cuda:0'),
  tensor([  18,   23,   29,  ...,    3, 2366, 3345], device='cuda:0')],
 [tensor([[  10, 1565,    6,  ...,    3, 2366, 3345],
          [   9,   57,  146,  ...,    2,    3,   14],
          [ 266,   31,  106,  ..

In [67]:
x,y = next(iter(data_lm.train_dl))
example = x[:20,:10].cpu()
texts = pd.DataFrame([data_lm.train_ds.vocab.textify(l).split(' ') for l in example])
texts

,0,1,2,3,4,5,6,7,8,9
0,xxfld,protagonist,xxunk,into,occasionally,start,humor,his,the,xxunk
1,1,is,for,this,xxunk,planning,is,revenge,box,in
2,un,xxunk,a,film,in,and,the,.,office,my
3,-,her,massive,",",other,not,biggest,still,",",xxunk
4,xxunk,early,series,although,versions,filming,problem,alive,xxunk,.
5,-,life,of,having,of,until,with,",",b.,first
6,believable,as,gags,the,the,everything,the,it,demille,","
7,!,a,built,main,story,has,film,looks,stopped,the
8,meg,butcher,upon,character,.,come,.,like,doing,xxunk
9,ryan,.,gags,a,wells,down,sure,carradine,films,scene


In [63]:
x[:,-1]

tensor([  27,   11,   80,    0,  389,    0,   30,   20, 1068,    0,   27, 1487,
          61,    0,   61,    8, 1156,    2,  138,    7,    0,    0,   30,   20,
           0, 4603,   27,    3,   19,   55,   26,  191,  182,  905,  116,   30,
           7,   80,   52,   32, 5834,   48,   27,   12,  264,  333,   23,   61,
         524,  905,   61,   30,    0,   27,    4,   61,  450,    7,    2, 1517,
          61,   30,    0,   27,    4,   61,   12, 5108,   23,    6,  905,   61,
          30,    0,   27,    4,   61,  398], device='cuda:0')

In [62]:
y.reshape((78,64))[:,-1]

tensor([  11,   80,    0,  389,    0,   30,   20, 1068,    0,   27, 1487,   61,
           0,   61,    8, 1156,    2,  138,    7,    0,    0,   30,   20,    0,
        4603,   27,    3,   19,   55,   26,  191,  182,  905,  116,   30,    7,
          80,   52,   32, 5834,   48,   27,   12,  264,  333,   23,   61,  524,
         905,   61,   30,    0,   27,    4,   61,  450,    7,    2, 1517,   61,
          30,    0,   27,    4,   61,   12, 5108,   23,    6,  905,   61,   30,
           0,   27,    4,   61,  398,    7], device='cuda:0')

In [ ]:
# download_wt103_model()

In [6]:
learn = RNNLearner.language_model(data_lm, pretrained_fnames=['lstm_wt103', 'itos_wt103'], drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

Total time: 00:17
epoch  train loss  valid loss  accuracy
0      4.727094    4.229310    0.245680  (00:17)



In [7]:
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3)

Total time: 01:44
epoch  train loss  valid loss  accuracy
0      4.502680    4.157105    0.250882  (00:21)
1      4.390687    4.073391    0.256216  (00:20)
2      4.271572    4.035617    0.260092  (00:21)
3      4.177719    4.019241    0.261403  (00:21)
4      4.113929    4.016751    0.260700  (00:20)



## Generate Text?

In [7]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize

[nltk_data] Downloading package punkt to /home/syang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [80]:
m.reset()

In [86]:
m = learn.model
s = word_tokenize('the thing')
t = LongTensor(data_lm.train_ds.vocab.numericalize(s)).view(-1,1).cuda()
t = torch.autograd.Variable(t)

In [87]:
# m.eval()
# res, *_ = m(t)
# # r = np.asarray(res[-1].detach().cpu())
# r = res[-1]
# sm = torch.nn.Softmax(dim=0)
# r = sm(r)
# r = np.asarray(r.detach().cpu())
# n = np.argmax(np.random.multinomial(1, r))
# word = data_lm.valid_ds.vocab.textify([n])
# n = LongTensor([n]).view(-1, 1).cuda()

In [88]:
# generation with multinomial
# there is some bug with torch library with torch.multinomial
# that throws and exception and make you restart the kernel
# reverting to numpy
m.eval()
for i in range(50):
    res, *_ = m(t)
    # r = np.asarray(res[-1].detach().cpu())
    r = res[-1]
    sm = torch.nn.Softmax(dim=0)
    r = sm(r)
    r = np.asarray(r.detach().cpu())
    n = np.argmax(np.random.multinomial(1, r))
    word = data_lm.valid_ds.vocab.textify([n])
    n = LongTensor([n]).view(-1, 1).cuda()
    print(word, end=' ')
    if word == '<eos>': break
    t = torch.cat((t, n))

is about this , since it 's xxunk . what , if you want real members of the society , in all of the supporting settings will take you " . since each member of the 

ValueError: sum(pvals[:-1]) > 1.0

In [71]:
# generation with top word
m.eval()
for i in range(100):
    res, *_ = m(t)
    n = res[-1].topk(10)[1]
    n = n[1] if n.data[0]==0 else n[0]
    word = data_lm.valid_ds.vocab.textify([n.item()])
    print(word, end=' ')
    if word == '<eos>': break
    t = torch.cat((t, n.unsqueeze(0).unsqueeze(0)))

been a matter of course , if not a matter of course , if the country is in a position to take a more active role in the country 's political and political life . the fact that the country is in a position to make a peace with the country is a matter of course . the country will be able to secure a peace with the country , and the country will be able to secure a peace with the country . the country will be able to secure a peace with the country , and the country 

In [140]:
m

SequentialRNN(
  (0): RNNCore(
    (encoder): Embedding(5999, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(5999, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1150)
      )
      (1): WeightDropout(
        (module): LSTM(1150, 1150)
      )
      (2): WeightDropout(
        (module): LSTM(1150, 400)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=5999, bias=True)
    (output_dp): RNNDropout()
  )
)

In [104]:
from nltk import word_tokenize
m = learn.model
s = word_tokenize('/n the big red dock')
t = LongTensor(data_lm.train_ds.vocab.numericalize(s)).view(-1,1).cuda()

In [118]:
for i in range(50):
    res,*_ = m(t)
    n=res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    word = data_lm.valid_ds.vocab.itos[n.item()]
    print(word, end=' ')
    if word == '<eos>': break
    t = torch.cat((t, n.unsqueeze(0).unsqueeze(0)))

- black . the black - white - white , white - white , and white - white - white , and the black - white - white , white - white , and white - collar white - collar white - collar white - collar white - collar white 